In [1]:
from json import load
from os import path
from itertools import chain, combinations
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr
from pymorphy2 import MorphAnalyzer

import numpy as np

m = MorphAnalyzer()

Using Theano backend.


In [ ]:
def get_word_vector(word_, model):
    if len(word_.split(' ')) == 1:
        if add_pos_tag:
            word = add_pos_tag_to_word(word_)
        else:
            word = word_
        try:
            return model[word]
        except KeyError:
            return np.zeros(shape=model.vector_size)
    else:
        vector = np.zeros(shape=model.vector_size)
        for subword in word_.split(' '):
            if add_pos_tag:
                word = add_pos_tag_to_word(subword)
            else:
                word = subword
            try:
                vector = np.add(vector, model[word])
            except KeyError:
                pass
        return vector / len(word_.split(' '))

In [ ]:
def add_pos_tag_to_word(word):
    return '{}_{}'.format(word, m.parse(word)[0].tag.POS)

## Load data

In [ ]:
words = load(open(path.join('..', 'data', 'test.json')))

In [ ]:
language = 'russian'
architecture = 'Word2Vec'
path_to_models = path.join('..', '..', '..', 'monolang', 'MODELS', architecture, language)
models_names = ['ruwikiruscorpora-superbigrams_skipgram_300_2_2018.vec']

In [ ]:
models = {}

for model_name in models_names:
    models[model_name] = KeyedVectors.load_word2vec_format(path.join(path_to_models, model_name))

In [ ]:
microframes_dict = {}
for word, anket in words.items():
    microframes_dict[word] = list(chain.from_iterable(anket.values()))
word_pairs = list(combinations(microframes_dict.keys(), 2))

## Evaluation typology 

In [ ]:
def get_vectors_distance(word1, word2, model, metric='cosine'):
    if metric == 'cosine':
        return 1 - cosine(model[word1], model[word2])

In [ ]:
def jaccard_distance(wordlist1, wordlist2):
    wordset1 = set(wordlist1)
    wordset2 = set(wordlist2)
    return float(len(wordset1 & wordset2)) / len(wordset1 | wordset2)

def sorensen_dice_distance(wordlist1, wordlist2):
    intersection = np.logical_and(wordlist1, wordlist2)
    return 2. * intersection.sum() / (wordlist1.sum() + wordlist2.sum())

def get_wordlist_similarity(wordlist1, wordlist2, metric='jaccard'):
    if metric == 'jaccard':
        return jaccard_distance(wordlist1, wordlist2)
    elif metric == 'sorensen':
        return sorensen_dice_distance(wordlist1, wordlist2)